https://scikit-learn.org/1.1/modules/clustering.html#k-means

## Training and Logging

In [1]:
from sklearn.datasets import load_iris
X, y = load_iris(return_X_y=True)
(X.shape, y.shape)

((150, 4), (150,))

In [2]:
(y[0], y[50], y[100])

(0, 1, 2)

In [3]:
import getpass

import mlflow
from liga.sklearn.mlflow import log_model
from sklearn.cluster import KMeans


mlflow_tracking_uri = "sqlite:///mlruns.db"
mlflow.set_tracking_uri(mlflow_tracking_uri)

# train a model
with mlflow.start_run() as run:
    ####
    # Part 1: Train the model and register it on MLflow
    ####
    model = KMeans(n_clusters=3, random_state=0)
    model.fit(X)
    registered_model_name = f"{getpass.getuser()}_kmeans"
    log_model(model, registered_model_name=registered_model_name)


/home/da/.cache/pants/named_caches/pex_root/venvs/d6a7829465a74d6f7ee1c45b5eb7d6677ce27680/cc443c29ee413022004e366aaeb20446296388d4/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/home/da/.cache/pants/named_caches/pex_root/venvs/d6a7829465a74d6f7ee1c45b5eb7d6677ce27680/cc443c29ee413022004e366aaeb20446296388d4/lib/python3.8/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
2023/01/06 11:12:55 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
Successfully registered model 'da_kmeans'.
2023/01/06 11:12:55 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.     

## Apply the model on large scale dataset

In [4]:
from example import spark
from rikai.spark.sql.codegen.mlflow_logger import CONF_MLFLOW_TRACKING_URI
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set(CONF_MLFLOW_TRACKING_URI, mlflow_tracking_uri)
spark.sql(f"""
CREATE OR REPLACE MODEL mlflow_sklearn_m USING 'mlflow:///{registered_model_name}';
"""
)

spark.sql("show models").show(1, vertical=False, truncate=False)


23/01/06 11:12:57 WARN Utils: Your hostname, debian resolves to a loopback address: 127.0.1.1; using 192.168.31.194 instead (on interface wlx1cbfce3ffbfe)
23/01/06 11:12:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/da/.cache/pants/named_caches/pex_root/installed_wheels/8f254bc20b539246427b2913639b8a0258db76ab54ba91fbbebb8dc8c36183c1/pyspark-3.3.1-py2.py3-none-any.whl/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/da/.ivy2/cache
The jars for the packages stored in: /home/da/.ivy2/jars
ai.eto#rikai_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e630a2b0-d191-4a04-a745-b3026022dc14;1.0
	confs: [default]
	found ai.eto#rikai_2.12;0.2.0-SNAPSHOT in local-ivy-cache
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found com.typesafe.scala-logging#scala-logging_2.12;3.9.4 in central
	found org.slf4j#slf4j-api;1.7.30 in spark-list
	found org.mlflow#mlflow-client;1.21.0 in central
	found org.apache.logging.log4j#log4j-core;2.17.1 in central
:: resolution report :: resolve 193ms :: artifacts dl 12ms
	:: modules in use:
	ai.eto#rikai_2.12;0.2.0-SNAPSHOT from local-ivy-cache in [default]
	com.typesafe.scala-logging#scala-logging_2.12;3.9.4 from central in [default]
	org.apache.logging.log4j#log4j-core;2.17.1 from central in [default]
	org.mlflow#mlflow-client;1.21.0 from central in [default]
	org.slf4j#slf4j-api;1.7.30 from

23/01/06 11:13:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
23/01/06 11:13:04 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
23/01/06 11:13:04 WARN SparkContext: The path file:///home/da/.ivy2/jars/ai.eto_rikai_2.12-0.2.0-SNAPSHOT.jar has been added already. Overwriting of added paths is not supported in the current version.
+-----

In [5]:
from liga.numpy.sql import literal

result = spark.sql(f"""
select
  ML_PREDICT(mlflow_sklearn_m, {literal(X[0])}) as y0,
  ML_PREDICT(mlflow_sklearn_m, {literal(X[50])}) as y50,
  ML_PREDICT(mlflow_sklearn_m, {literal(X[100])}) as y100
"""
)

result.printSchema()
result.toPandas()

root
 |-- y0: integer (nullable = true)
 |-- y50: integer (nullable = true)
 |-- y100: integer (nullable = true)



,y0,y50,y100
0,1,0,2
